**Setup and Installation**

In [ ]:
!pip install langchain langchain-openai tiktoken faiss-cpu

import os
import re
import json
import tiktoken
from typing import List, Dict, Any

os.environ["OPENAI_API_KEY"] = "your-api-key"

from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain_core.documents import Document

**Basic Utility Functions**

In [2]:
def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """Count the number of tokens in a text string."""
    encoder = tiktoken.encoding_for_model(model)
    return len(encoder.encode(text))

def print_separator():
    """Print a visual separator."""
    print("\n" + "="*50 + "\n")

**Section 1: Variable Substitution**

In [ ]:
print("Section 1: Variable Substitution - Basic Template Example")

template = """
Answer the question based on the context below.

Context: {context}

Question: {question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

# Example usage
formatted_prompt = prompt.format(
    context="The capital of France is Paris. It is known for the Eiffel Tower.",
    question="What is the capital of France?"
)

print(formatted_prompt)
print_separator()

**Section 2: Programmatic Template Selection**

In [ ]:
print("Section 2: Programmatic Template Selection")

# Define different template types
qa_template = """
Answer the question based solely on the provided context.

Context: {context}

Question: {question}

Answer:
"""

comparison_template = """
Compare and contrast the following entities based on the provided context.

Context: {context}

Entities to compare: {entities}

Comparison:
"""

summarization_template = """
Summarize the following information.

Context: {context}

Summary:
"""

def select_template(query: str) -> PromptTemplate:
    """Select the appropriate template based on query analysis."""
    comparison_terms = ["compare", "difference", "versus", "vs", "similarities", "different"]
    summarization_terms = ["summarize", "summary", "overview", "brief"]

    # Convert templates to PromptTemplate objects
    templates = {
        "qa": PromptTemplate(
            input_variables=["context", "question"],
            template=qa_template
        ),
        "comparison": PromptTemplate(
            input_variables=["context", "entities"],
            template=comparison_template
        ),
        "summarization": PromptTemplate(
            input_variables=["context"],
            template=summarization_template
        )
    }

    # Template selection logic
    if any(term in query.lower() for term in comparison_terms):
        print(f"Selected template: comparison")
        return templates["comparison"]
    elif any(term in query.lower() for term in summarization_terms):
        print(f"Selected template: summarization")
        return templates["summarization"]
    else:
        print(f"Selected template: question-answering")
        return templates["qa"]

# Test with different queries
test_queries = [
    "What is the capital of France?",
    "Compare the benefits of Python and JavaScript",
    "Summarize the key points about climate change"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    template = select_template(query)
    print(f"Template variables: {template.input_variables}")

print_separator()

**Section 3: Context Window Management**

In [ ]:
print("Section 3: Context Window Management")

def fit_documents_to_token_limit(
    docs: List[Document],
    token_limit: int,
    model: str = "gpt-3.5-turbo"
) -> str:
    """Process documents to fit within token limit."""
    processed_docs = []
    current_tokens = 0

    for i, doc in enumerate(docs):
        # Add document separator
        doc_text = f"\n[Document {i+1}]: {doc.page_content}\n"
        doc_tokens = count_tokens(doc_text, model)

        if current_tokens + doc_tokens <= token_limit:
            processed_docs.append(doc_text)
            current_tokens += doc_tokens
        else:
            # If the first document is already too large, truncate it
            if i == 0:
                encoder = tiktoken.encoding_for_model(model)
                tokens = encoder.encode(doc_text)
                truncated_tokens = tokens[:token_limit]
                truncated_text = encoder.decode(truncated_tokens)
                processed_docs.append(truncated_text)
                current_tokens = token_limit
                break
            else:
                # Otherwise, we've reached our limit with previous docs
                break

    return "".join(processed_docs)

def create_dynamic_prompt(
    query: str,
    retrieved_docs: List[Document],
    max_tokens: int = 3500,
    model: str = "gpt-3.5-turbo"
) -> str:
    """Create a dynamic prompt, fitting retrieved docs into the token limit."""
    # Basic template parts
    system_instruction = "You are an assistant that answers based on context."
    query_section = f"Question: {query}"
    answer_section = "Answer:"

    # Calculate tokens for the base template
    template_skeleton = f"{system_instruction}\n\nContext: [PLACEHOLDER]\n\n{query_section}\n\n{answer_section}"
    base_tokens = count_tokens(template_skeleton.replace("[PLACEHOLDER]", ""), model)

    # Calculate available tokens for context
    available_context_tokens = max_tokens - base_tokens

    # Process documents to fit within available tokens
    processed_docs = fit_documents_to_token_limit(retrieved_docs, available_context_tokens, model)

    # Construct the final prompt
    prompt = template_skeleton.replace("[PLACEHOLDER]", processed_docs)

    return prompt

# Sample documents for testing
sample_docs = [
    Document(page_content="Paris is the capital of France. It is known for the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral."),
    Document(page_content="France is a country in Western Europe. Its capital is Paris. France is known for its cuisine, culture, and history."),
    Document(page_content="The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after engineer Gustave Eiffel.")
]

# Test the dynamic prompt construction
query = "What is the capital of France and what is it known for?"
dynamic_prompt = create_dynamic_prompt(query, sample_docs, max_tokens=1000)
print(f"Dynamic prompt (token count: {count_tokens(dynamic_prompt)}):")
print(dynamic_prompt)

print_separator()

**Section 4: Document Prioritization**

In [ ]:
print("Section 4: Document Prioritization")

def add_relevance_scores(docs: List[Document], query: str) -> List[Document]:
    """
    Simulate relevance scoring for documents.
    In a real system, this would come from the vector store retrieval.
    """
    scored_docs = []

    for doc in docs:
        # Simple mock relevance calculation (just for demonstration)
        relevance = 0.0
        doc_text = doc.page_content.lower()
        query_terms = query.lower().split()

        for term in query_terms:
            if term in doc_text:
                relevance += 0.2 + (doc_text.count(term) * 0.05)

        # Ensure the score is between 0 and 1
        relevance = min(max(relevance, 0.1), 1.0)

        # Add the score to the document metadata
        metadata = doc.metadata.copy() if hasattr(doc, 'metadata') else {}
        metadata['relevance_score'] = relevance

        # Create a new document with the updated metadata
        scored_doc = Document(page_content=doc.page_content, metadata=metadata)
        scored_docs.append(scored_doc)

    return scored_docs

def prioritize_documents(docs: List[Document]) -> List[Document]:
    """Prioritize documents based on relevance score."""
    if not docs:
        return []

    # Check if documents have relevance scores
    if 'relevance_score' not in docs[0].metadata:
        return docs  # Return original if no scores

    # Sort by relevance score (descending)
    sorted_docs = sorted(docs, key=lambda x: x.metadata.get('relevance_score', 0), reverse=True)

    # Group documents by relevance score
    high_relevance = [doc for doc in sorted_docs if doc.metadata.get('relevance_score', 0) > 0.7]
    medium_relevance = [doc for doc in sorted_docs if 0.4 <= doc.metadata.get('relevance_score', 0) <= 0.7]
    low_relevance = [doc for doc in sorted_docs if doc.metadata.get('relevance_score', 0) < 0.4]

    # Combine with priority weighting
    return high_relevance + medium_relevance + low_relevance

# Test document prioritization
query = "What is Paris known for?"
scored_docs = add_relevance_scores(sample_docs, query)

print("Documents with relevance scores:")
for i, doc in enumerate(scored_docs):
    print(f"Document {i+1} (score: {doc.metadata.get('relevance_score', 0):.2f}):")
    print(f"  {doc.page_content}")

prioritized_docs = prioritize_documents(scored_docs)

print("\nPrioritized documents:")
for i, doc in enumerate(prioritized_docs):
    print(f"Document {i+1} (score: {doc.metadata.get('relevance_score', 0):.2f}):")
    print(f"  {doc.page_content}")

print_separator()

**Section 5: Content Transformation**

In [ ]:
print("Section 5: Content Transformation")

def extract_relevant_passages(doc: Document, query: str, max_length: int = 200) -> str:
    """
    Extract the most relevant passage from a document.
    In a real system, this might use more sophisticated techniques.
    """
    if not doc or not doc.page_content:
        return ""

    text = doc.page_content

    # If text is already short enough, return it all
    if len(text) <= max_length:
        return text

    # Simple approach: find query terms in text
    query_terms = set(query.lower().split())
    best_passage = ""
    highest_term_count = 0

    # Create overlapping windows of text
    window_size = min(max_length, len(text))
    step_size = max(window_size // 4, 1)

    for i in range(0, len(text) - window_size + 1, step_size):
        passage = text[i:i + window_size]
        passage_lower = passage.lower()
        term_count = sum(1 for term in query_terms if term in passage_lower)

        if term_count > highest_term_count:
            highest_term_count = term_count
            best_passage = passage

    # If no terms found, take the beginning of the document
    if not best_passage:
        best_passage = text[:max_length]

    return best_passage

def transform_document(doc: Document, query: str) -> str:
    """Transform a document for inclusion in a prompt."""
    if not doc:
        return ""

    # Extract relevant passage
    relevant_passage = extract_relevant_passages(doc, query)

    # Add source metadata
    doc_id = doc.metadata.get('doc_id', 'unknown')
    source = doc.metadata.get('source', 'unknown')

    return f"[Document {doc_id} from {source}]:\n{relevant_passage}"

# Add some metadata to our test documents
for i, doc in enumerate(sample_docs):
    doc.metadata = {'doc_id': i+1, 'source': f'source_{i+1}'}

# Test content transformation
query = "What is the Eiffel Tower?"
for doc in sample_docs:
    transformed_content = transform_document(doc, query)
    print(transformed_content)
    print("-" * 30)

print_separator()

**Section 6: Adaptive Instruction Calibration**

In [ ]:
print("Section 6: Adaptive Instruction Calibration")

def retrieval_confidence(docs: List[Document]) -> float:
    """Calculate overall confidence in retrieved documents."""
    if not docs:
        return 0.0

    # Get average relevance score
    scores = [doc.metadata.get('relevance_score', 0) for doc in docs]
    avg_score = sum(scores) / len(scores) if scores else 0

    return avg_score

def requires_reasoning(query: str) -> bool:
    """Determine if a query requires explicit reasoning."""
    reasoning_indicators = [
        "why", "how", "explain", "reason", "analyze", "compare",
        "evaluate", "synthesize", "interpret", "assess"
    ]

    return any(indicator in query.lower() for indicator in reasoning_indicators)

def calibrate_instructions(query: str, retrieved_docs: List[Document]) -> str:
    """Dynamically calibrate system instructions based on query and retrieval."""
    # Base instructions
    instructions = "Answer based on the context provided."

    # Assess retrieval quality
    confidence = retrieval_confidence(retrieved_docs)

    if confidence < 0.5:
        instructions += " If the context doesn't contain sufficient information, clearly state what's missing."

    # Analyze query complexity
    if requires_reasoning(query):
        instructions += " Break down your reasoning process step by step."

    # Check for numerical/factual questions
    if any(word in query.lower() for word in ["how many", "when", "where", "who", "which"]):
        instructions += " Be precise with factual information."

    # Check for definitional questions
    if query.lower().startswith("what is") or "define" in query.lower():
        instructions += " Provide a clear, concise definition before elaborating."

    return instructions

# Test instruction calibration
test_queries = [
    "What is the population of Paris?",
    "Why is the Eiffel Tower significant?",
    "Where is the Louvre Museum located?",
    "How does French culture influence its cuisine?"
]

for query in test_queries:
    print(f"Query: {query}")
    instructions = calibrate_instructions(query, scored_docs)
    print(f"Calibrated instructions: {instructions}")
    print("-" * 30)

print_separator()

**Section 7: Putting It All Together**

In [ ]:
print("Section 7: Putting It All Together")

# This function integrates all the components
def build_dynamic_rag_prompt(query: str, docs: List[Document], max_tokens: int = 3500) -> str:
    """
    Comprehensive function to build a dynamic RAG prompt:
    1. Score and prioritize documents
    2. Transform documents for inclusion
    3. Calibrate instructions
    4. Fit within token limits
    """
    # If no API key, just show the process without making API calls
    if not os.environ.get("OPENAI_API_KEY"):
        print("No OpenAI API key found - demonstrating prompt building only")

    # 1. Score and prioritize documents
    if 'relevance_score' not in (docs[0].metadata if docs else {}):
        scored_docs = add_relevance_scores(docs, query)
    else:
        scored_docs = docs

    prioritized_docs = prioritize_documents(scored_docs)

    # 2. Transform documents
    transformed_docs = [
        transform_document(doc, query) for doc in prioritized_docs
    ]

    # 3. Calibrate instructions
    system_instruction = calibrate_instructions(query, scored_docs)

    # 4. Calculate token budgets
    query_section = f"Question: {query}"
    answer_section = "Answer:"

    # Calculate tokens for the base template
    template_skeleton = f"{system_instruction}\n\nContext: [PLACEHOLDER]\n\n{query_section}\n\n{answer_section}"
    base_tokens = count_tokens(template_skeleton.replace("[PLACEHOLDER]", ""))

    # Calculate available tokens for context
    available_context_tokens = max_tokens - base_tokens

    # 5. Fit documents within token limit
    context_text = ""
    current_tokens = 0

    for doc_text in transformed_docs:
        doc_tokens = count_tokens(doc_text)
        if current_tokens + doc_tokens <= available_context_tokens:
            context_text += doc_text + "\n\n"
            current_tokens += doc_tokens
        else:
            break

    # 6. Construct the final prompt
    final_prompt = template_skeleton.replace("[PLACEHOLDER]", context_text.strip())

    return final_prompt

# Test the integrated approach
query = "Why is the Eiffel Tower important to Paris and what are its key features?"
final_prompt = build_dynamic_rag_prompt(query, sample_docs, max_tokens=2000)

print("Final dynamic RAG prompt:")
print(final_prompt)
print(f"\nTotal token count: {count_tokens(final_prompt)}")

print_separator()

**Bonus: LangChain Implementation Example**

In [ ]:
print("Bonus: LangChain Implementation Example")

# Note: This will only run if you have set your OpenAI API key
try:
    llm = OpenAI(temperature=0)

    def generate_rag_response(query: str, docs: List[Document]) -> str:
        """Generate a response using a dynamic RAG prompt."""
        # Build dynamic prompt
        prompt_text = build_dynamic_rag_prompt(query, docs)

        # Create prompt template
        prompt = PromptTemplate.from_template("{prompt}")

        # Create chain
        chain = LLMChain(llm=llm, prompt=prompt)

        # Run chain
        response = chain.run(prompt=prompt_text)

        return response

    # Only execute if API key is available
    if os.environ.get("OPENAI_API_KEY"):
        print("Using OpenAI to generate response...")
        response = generate_rag_response(
            "What features make the Eiffel Tower unique?",
            sample_docs
        )
        print("\nGenerated Response:")
        print(response)
    else:
        print("OpenAI API key not set. Skipping response generation.")
except Exception as e:
    print(f"Error in LangChain implementation: {e}")

print_separator()

print("Notebook completed!")